In [ ]:
import pandas as pd
import numpy as np
from scipy.io.wavfile import write
from scipy.signal import chirp
import matplotlib.pyplot as plt
import re
from pydub import AudioSegment

In [ ]:
def parse_log_file(file_path):
    parsed_data = []
    with open(file_path, 'r') as f:
        for line_num, line in enumerate(f, start=1):
            parts = line.strip().split(',')
            if len(parts) < 7:
                continue
            try:
                timestamp = parts[0].split()[1] if len(parts[0].split()) > 1 else None
                event_type = parts[1]
                source = parts[2] if len(parts) > 2 else None
                target = parts[6] if len(parts) > 6 else None
                spell_id = parts[10] if len(parts) > 10 else None
                spell_name = parts[11] if len(parts) > 11 else None
                parsed_data.append([timestamp, event_type, source, target, spell_id, spell_name])
            except Exception as e:
                continue
    
    columns = ['Timestamp', 'EventType', 'Source', 'Target', 'SpellID', 'SpellName']
    df = pd.DataFrame(parsed_data, columns=columns)
    
    df['Seconds'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S.%f', errors='coerce')
    df = df.dropna(subset=['Seconds'])
    df['Seconds'] = (df['Seconds'] - df['Seconds'].iloc[0]).dt.total_seconds()
    return df

In [ ]:
def generate_frequency_mapping(event_types):
    base_frequency = 220
    frequency_mapping = {}
    for i, event in enumerate(event_types):
        frequency_mapping[event] = base_frequency * (2 ** (i / 12))
    return frequency_mapping

def generate_sine_wave(freq, duration, sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = np.sin(2 * np.pi * freq * t)
    return wave

def compress_events(df, total_duration=60):
    unique_seconds = df['Seconds'].max() - df['Seconds'].min()
    compression_ratio = unique_seconds / total_duration
    df['CompressedTime'] = df['Seconds'] / compression_ratio
    return df

In [ ]:
def calculate_dynamic_duration(row, min_duration=0.5, max_duration=1.5):
    try:
        base_value = int(row['SpellID']) % 10 if row['SpellID'] and row['SpellID'].isdigit() else 5
    except ValueError:
        base_value = 5
    duration = min_duration + (base_value / 10) * (max_duration - min_duration)
    return duration

In [ ]:
def main(log_file_path):
    df = parse_log_file(log_file_path)
    df = compress_events(df)
    
    unique_events = df['EventType'].unique()
    frequency_mapping = generate_frequency_mapping(unique_events)
    
    sample_rate = 44100
    total_duration = 60
    audio_length = sample_rate * total_duration
    audio = np.zeros(audio_length)

    for _, row in df.iterrows():
        freq = frequency_mapping.get(row['EventType'], 440)
        duration = calculate_dynamic_duration(row)
        wave = generate_sine_wave(freq, duration, sample_rate)

        if len(wave) == 0:
            continue

        start_sample = int(row['CompressedTime'] * sample_rate)
        end_sample = start_sample + len(wave)

        if start_sample < 0 or end_sample <= start_sample:
            continue
        if end_sample > len(audio):
            wave = wave[:len(audio) - start_sample]

        try:
            audio[start_sample:start_sample + len(wave)] += wave
        except ValueError as e:
            print(f"Invalid Range: st={start_sample}, end={end_sample}, wav={len(wave)}")
            continue
    
    if np.max(np.abs(audio)) > 0:
        audio = audio / np.max(np.abs(audio))
    else:
        print("Warning: Audio data is empty after processing.")
        return

    
    audio = (audio * 32767).astype(np.int16)
    wav_output_file = "result_audio.wav"
    write(wav_output_file, sample_rate, audio)

    mp3_output_file = "result_audio.mp3"
    sound = AudioSegment.from_wav(wav_output_file)
    sound.export(mp3_output_file, format="mp3")

In [ ]:
main("WowCombatLog.txt")